In [ ]:
!pip install texthero

     |████████████████████████████████| 245kB 11.5MB/s 
     |████████████████████████████████| 1.4MB 18.9MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434675 sha256=be80d292a18e61495d87fb68e499563a5c2b0539bcdc4d4b35da79b6f49a1a71
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
import texthero as hero
from scipy.sparse import hstack

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
ROOT_PATH = "/content/drive/My Drive/Magíster 2020"

In [ ]:
puc_train = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/puc_train_70.json", orient="records")
puc_test = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/puc_test_30.json", orient="records")

puj_train = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/puj_train_70.json", orient="records")
puj_test = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/puj_test_30.json", orient="records")

aurora_train = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/aurora_train_70.json", orient="records")
aurora_test = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/aurora_test_30.json", orient="records")

train = pd.concat([puc_train, puj_train, aurora_train], ignore_index=True)
test = pd.concat([puc_test, puj_test, aurora_test], ignore_index=True)

In [ ]:
train["title"] = hero.clean(train["title"])
test["title"] = hero.clean(test["title"])

In [ ]:
train["clean_abstract"] = hero.clean(train["clean_abstract"])
test["clean_abstract"] = hero.clean(test["clean_abstract"])

In [ ]:
feature_extraction_titles = TfidfVectorizer()

In [ ]:
X_titles_train = feature_extraction_titles.fit_transform(train["title"]).toarray()
X_titles_test = feature_extraction_titles.transform(test["title"]).toarray()

In [ ]:
pca_titles = PCA(n_components=512, svd_solver='full')

In [ ]:
X_titles_train = pca_titles.fit_transform(X_titles_train)
X_titles_test = pca_titles.transform(X_titles_test)

In [ ]:
feature_extraction_abstracts = TfidfVectorizer()

In [ ]:
X_abstracts_train = feature_extraction_abstracts.fit_transform(train["clean_abstract"]).toarray()
X_abstracts_test = feature_extraction_abstracts.transform(test["clean_abstract"]).toarray()

In [ ]:
pca_abstracts = PCA(n_components=512, svd_solver='full')

In [ ]:
X_abstracts_train = pca_abstracts.fit_transform(X_abstracts_train)
X_abstracts_test = pca_abstracts.transform(X_abstracts_test)

In [ ]:
X_train = np.hstack([X_titles_train, X_abstracts_train])
X_test = np.hstack([X_titles_test, X_abstracts_test])

In [ ]:
clf = SVC(probability=True, kernel='rbf')
clf.fit(X_train, train["first_sdg"])

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(test["first_sdg"], y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        13
           2       0.55      0.24      0.33        71
           3       0.75      0.93      0.83       771
           4       0.83      0.80      0.81       201
           5       0.75      0.67      0.71       122
           6       0.63      0.73      0.68        91
           7       0.66      0.70      0.68       109
           8       0.75      0.19      0.30        32
           9       0.42      0.46      0.44       136
          10       0.69      0.25      0.37        79
          11       0.68      0.57      0.62       137
          12       0.41      0.25      0.31       106
          13       0.55      0.47      0.51       116
          14       0.62      0.49      0.55        59
          15       0.54      0.68      0.60       149
          16       0.63      0.76      0.69       109
          17       0.67      0.13      0.22        45

    accuracy              

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

